# Beginner's Guide to XGBoost for Classification Problems
## Utilize the hottest ML library for state-of-the-art performance
<img src='images/pexels.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@dom-gould-105501?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Dom Gould</a>
        on 
        <a href='https://www.pexels.com/photo/rear-view-of-silhouette-man-against-sky-during-sunset-325790/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Setup

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb

### What is XGBoost and why is it so popular?

Let me introduce you to the hottest Machine Learning library in the ML community - XGBoost. In recent years, it has been the main driving force behind the algorithms that win massive ML competitions. Its speed and performance is unparalleled and it consistently outperforms any other algorithms aimed at supervised learning tasks.

The library is parallelizable which means the core algorithm can run on clusters of GPUs or even across a network of computers. This makes it feasible to solve ML tasks by training on hundreds of millions of training examples with high performance. 

Originally, it was written in C++ as a command line application. After winning a huge competition in the field of physics, it started being widely adopted by the ML community. As a result, now the library has its APIs in several other languages including Python, R and Julia. 

In this post, we will learn the fundamentals of XGBoost by solving classification tasks. ## TODO

### Refresher on Terminology

Before we move on to code examples of XGBoost, let's refresh on some of the terms we will be using throughout the post. 

**Classification task**: a supervised machine learning task in which one should predict if an instance is in some category by studying the item's features. For example, by looking at the body measurements, patient history and glucose levels of a person, you can predict whether a person belongs to 'Has diabetes' or 'Does not have diabetes' group.

**Binary classification**: One type of classification where the target instance can only belong to either one of two classes. For example, predicting whether an email is spam or not, whether a customer purchases some product or not, etc.

**Multi-class classification**: Another type of classification problems where the target can belong to one of many categories. For example, predicting the species of a bird, guessing someones bloody type, etc.

If you find yourself confused by other terminology, I have written a small ML dictionary for beginners:

https://towardsdatascience.com/codeless-machine-learning-dictionary-for-dummies-fa912cc7bdfe

### How to prepocess your datasets for XGBoost

Apart from basic data cleaning operations, there are some requirements for XGBoost to achieve top performance. Mainly:
- Numeric features should be scaled
- Categorical features should be encoded

To show how these steps are done, we will be using the [Rain in Australia](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) dataset from Kaggle where we will predict whether it will rain today or not based on some weather measurements. In this section, we will focus on preprocessing by utilizing Scikit-Learn Pipelines.

In [3]:
import pandas as pd

rain = pd.read_csv("data/weatherAUS.csv")
rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

The dataset contains weather measures of 10 years from multiple weather stations in Australia. You can either predict whether it will rain tomorrow or today, so there are two features in the dataset named `RainToday`, `RainTomorrow`.

Since we will only be predicting for `RainToday`, we will drop the other one along with some other features that won't be necessary:

In [7]:
cols_to_drop = ["Date", "Location", "RainTomorrow"]

rain.drop(cols_to_drop, axis=1, inplace=True)

Next, let's deal with missing values starting by looking at their proportions in each column:

In [13]:
missing_props = rain.isna().mean(axis=0)

missing_props

MinTemp          0.010209
MaxTemp          0.008669
Rainfall         0.022419
Evaporation      0.431665
Sunshine         0.480098
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Cloud3pm         0.408071
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
dtype: float64

If the proportion is higher than 40% we will drop the column:

In [18]:
over_threshold = missing_props[missing_props >= 0.4]

over_threshold

Evaporation    0.431665
Sunshine       0.480098
Cloud3pm       0.408071
dtype: float64

Three columns contain more than 40% missing values:

In [22]:
rain.drop(over_threshold.index, axis=1, inplace=True)

Next, there are both categorical and numeric features. We will build two separate pipelines and combine them later. 

> The next code examples will heavily use Sklearn-Pipelines. If you are not familiar with them, check out my separate article for the [complete guide](https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d) on them.

For the categorical features, we will impute the missing values with the mode of the column and encode them with One-Hot encoding:

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('oh-encode', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

For the numeric features, I will choose the mean as an imputer and `StandardScaler` so that the features has 0 mean and a variance of 1:

In [26]:
from sklearn.preprocessing import StandardScaler

numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

Finally, we will combine the two pipelines with a column transformer. To specify which columns the pipelines are designed for, we should first isolate the categorical and numeric feature names:

In [31]:
cat_cols = rain.select_dtypes(exclude='number')
num_cols = rain.select_dtypes(include='number')

Next, we will input these along with their corresponding pipelines into a `ColumnTransFormer` instance:

In [32]:
from sklearn.compose import ColumnTransformer

full_processor = ColumnTransformer(transformers=[
    ('numeric', numeric_pipeline, num_cols),
    ('categorical', categorical_pipeline, cat_cols)
])

The full pipeline is finally ready. The only thing missing is the XGBoost classifier, which we will add in the next section.

### An Example of XGBoost For a Classification Problem

### What Powers XGBoost Under the Hood

### Cross-validation OR Hyperparameter Tuning